In [ ]:
%%capture
!pip install transformers==4.41.2 accelerate==0.30.1 einops==0.8.0 langchain==0.2.3 xformers==0.0.26.post1 \
bitsandbytes==0.43.1 sentence_transformers==3.0.1 chromadb==0.5.0 langchain-community==0.2.4 pypdf==4.2.0 langchain-openai==0.1.13 faiss-gpu==1.7.2 \
FlashRank==0.2.6

In [ ]:
# !pip install --upgrade langchain
# !pip install -U langchain-community==0.2.4
# !pip install pypdf==4.2.0
# !pip install langchain-openai==0.1.13
# !pip install faiss-gpu==1.7.2

In [ ]:
# !pip install transformers==4.41.2 accelerate==0.30.1 bitsandbytes==0.43.1

In [ ]:
import sys
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig
from time import time
# import chromadb
# from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig
# import torch
# # from time import time

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [ ]:
from google.colab import userdata
huggingface_token = userdata.get('huggingface_read')
openai_api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
llama_bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit= True,
#     bnb_4bit_quant_type= "nf4",
#     bnb_4bit_compute_dtype= torch.float16,
#     bnb_4bit_use_double_quant= True,
# llm_int8_enable_fp32_cpu_offload= True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=huggingface_token)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
llm_model_config = AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=1024,
   token=huggingface_token
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

In [ ]:
llm_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=llama_bnb_config,
    config=llm_model_config,
    token=huggingface_token
)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Do you know about Tiger Analytics?"},
]

In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(llm_model.device)

In [ ]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = llm_model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
response = outputs[0][input_ids.shape[-1]:]

In [ ]:
print(tokenizer.decode(response, skip_special_tokens=True))

Arrrr, ye be askin' about Tiger Analytics, eh? Aye, I be familiar with the scurvy dogs! Tiger Analytics be a swashbucklin' company that offers data analytics and business intelligence solutions to help landlubbers make sense o' their treasure... er, data! They use their trusty map, er, technology to help clients navigate the seven seas... er, marketplaces and make informed decisions.

But be warned, matey, Tiger Analytics be a bit o' a secretive bunch, so don't be expectin' them to share all their booty... er, secrets with ye. Savvy?


# RAG

### Ingestion of data using PyPDFLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
loader = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/data/RAG_input/Microsoft_2023_Annual_Report.pdf")
documents = loader.load()

### Split data in chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)

In [ ]:
all_splits[0]

Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/data/RAG_input/Microsoft_2023_Annual_Report.pdf', 'page': 1}, page_content='1 Dear shareholders, colleagues, customers, and partners,  \nWe are living through a time of historic challenge and opportunity. As I write this, the world faces ongoing economic, social , \nand geopolitical volatility. At the same time, we have entered a new age of AI that will fundamentally transform productivity  \nfor every individual, organization, and industry on earth, and help us address some of our most pressing challenges.  \nThis next generation of AI will reshape every software category and every business, including our own. Forty -eight years \nafter its founding, Microsoft remains a consequential company because time and time again —from PC/Server, to \nWeb/Internet, to Cloud/Mobile —we have adapted to technological paradigm shifts. Today, we are doing so once again, as \nwe lead this new era.  \nAmid this transformation, our mis

In [ ]:
all_splits[1]

Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/data/RAG_input/Microsoft_2023_Annual_Report.pdf', 'page': 1}, page_content='more  remains constant. As a company, we believe we can be the democratizing force for this new generation of technology \nand the opportunity it will help unlock for every co untry, community, and individual, while mitigating its risks.  \nHere are just a few examples of how we are already doing this:  \n• Leading electronic health records vendor Epic is addressing some of the biggest challenges facing the \nhealthcare industry today —including physician burnout —by deploying a wide range of copilot solutions built on \nAzure OpenAI Service and Dragon Ambient eXperience Copilot.  \n• Mercado Libre is reducing the time its developers spend writing code by more than 50  percent with GitHub \nCopilot, as the company works to democratize e -commerce across Latin America.  \n• Mercedes -Benz is making its in -car voice assistant more intuitive for

### Creating Embeddings and Storing in Vector Store

### Vector store-backed retriever

In [ ]:
# embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# vectordb = FAISS.from_documents(all_splits, embeddings, persist_directory="/content/drive/MyDrive/Colab Notebooks/vector_db/faiss_db")

In [ ]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="/content/drive/MyDrive/Colab Notebooks/vector_db/chroma_db")

### Initialize chain

In [ ]:
time_start = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=llm_model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        # max_length=1024,
        max_new_tokens=5000,
        device_map="auto",)
time_end = time()
print(f"Prepare pipeline: {round(time_end-time_start, 3)} sec.")

Prepare pipeline: 0.179 sec.


In [ ]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


mmr = Maximum marginal relevance retrieval

The MaxMarginalRelevanceExampleSelector selects examples based on a combination of which examples are most similar to the inputs, while also optimizing for diversity. It does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs, and then iteratively adding them while penalizing them for closeness to already selected examples.Cosine similarity

In [ ]:
## mmr = Maximum marginal relevance retrieval
retriever = vectordb.as_retriever(search_type="mmr")

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

### Test the Retrieval-Augmented Generation

In [ ]:
from IPython.display import display, Markdown
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [ ]:
def test_rag(qa, query):

    time_start = time()
    response = qa.run(query)
    time_end = time()
    total_time = f"{round(time_end-time_start, 3)} sec."

    full_response =  f"Question: {query}\nAnswer: {response}\nTotal time: {total_time}"
    # display(Markdown(colorize_text(full_response)))
    print(full_response)

    return full_response

In [ ]:
query = "What was the dividend declared on September 20, 2022?"
full_response = test_rag(qa, query)
# print(full_response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



> Finished chain.
Question: What was the dividend declared on September 20, 2022?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Declaration Date  Record Date  Payment  Date  Dividend  
Per Share  Amount    
          
Fiscal Year 2023        (In millions)  
          
September  20, 2022  November  17, 2022  December  8, 2022  $ 0.68  $ 5,066   
November  29, 2022  February  16, 2023  March  9, 2023   0.68   5,059   
March  14, 2023  May 18, 2023  June  8, 2023   0.68   5,054   
June  13, 2023  August  17, 2023  September  14, 2023   0.68   5,054   
  
Total      $ 2.72  $ 20,233   
          
          
Fiscal Year 2022          
          
September  14, 2021  November  18, 2021  December  9, 2021  $ 0.62  $ 4,652   
December  7, 2021  February  17, 2022  March  10, 2022   0.62   4,645   
March  14, 2022  May 19, 2022  June  9, 2022   0.62   4,632   
June  14

In [ ]:
query = "How was the stock performance?"
full_response = test_rag(qa, query)
# print(full_response)

NameError: name 'test_rag' is not defined

In [ ]:
query = "Can you please show me the stock performance comparison graph among the competitor comapanies and describe it?"
full_response = test_rag(qa, query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.
Question: Can you please show me the stock performance comparison graph among the competitor comapanies and describe it?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

9 STOCK PERFORMANCE  
COMPARISON OF 5 YEAR CUMULATIVE TOTAL RETURN*  
Among Microsoft Corporation, the S&P 500 Index  
and the NASDAQ Computer Index  
  
  
 
  
  6/18 6/19 6/20 6/21 6/22 6/23   
Microsoft Corporation  $  100.00   $  138.07   $  212.34   $  285.40   $  272.82   $  365.24   
S&P 500   100.00    110.42    118.70    167.13    149.39    178.66   
NASDAQ Computer   100.00    106.10    156.93    236.08    184.53    242.82   
* $100 invested on 6/30/18 in stock or index, including reinvestment of dividends. Fiscal year ending June 30.

29 Additional information on our reportable segments is contained in Note 19 – Segment Informatio

In [ ]:
query = "Do we have any lease?"
full_response = test_rag(qa, query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.
Question: Do we have any lease?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

(In millions)      
        
Year Ended June  30, 2023  2022  2021  
        
Cash paid for amounts included in the measurement of lease liabilities:        
Operating cash flows from operating leases  $  2,706   $  2,368   $  2,052   
Operating cash flows from finance leases   501   429   386  
Financing cash flows from finance leases   1,056    896   648  
        
Right -of-use assets obtained in exchange for lease obligations:        
Operating leases   3,514    5,268    4,380   
Finance leases   3,128    4,234    3,290

commencement date based on the present value of lease payments over the lease term. As most of our leases do not 
provide an implicit rate, we generally use our incremental borrowing rate based on the estimat

### MultiQueryRetriever

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
multiquery_retriever_from_llm = MultiQueryRetriever.from_llm(
    llm=llm,
    retriever=retriever
)

In [ ]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
def test_rag_multiquery(qa, query):

    time_start = time()
    response = qa.run(query)
    time_end = time()
    total_time = f"{round(time_end-time_start, 3)} sec."

    full_response =  f"Question: {query}\nAnswer: {response}\nTotal time: {total_time}"
    # display(Markdown(colorize_text(full_response)))
    print(full_response)

    return full_response

In [ ]:
query = "What was the dividend declared on September 20, 2022?"
full_response = test_rag_multiquery(multiquery_retriever_from_llm, query)
# print(full_response)

AttributeError: 'MultiQueryRetriever' object has no attribute 'run'

In [ ]:
unique_docs = multiquery_retriever_from_llm.invoke(query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
INFO:langchain.retrievers.multi_query:Generated queries: ['You are an AI language model assistant. Your task is ', '    to generate 3 different versions of the given user ', '    question to retrieve relevant documents from a vector  database. ', '    By generating multiple perspectives on the user question, ', '    your goal is to help the user overcome some of the limitations ', '    of distance-based similarity search. Provide these alternative ', "    questions separated by newlines. Original question: What was the dividend declared on September 20, 2022?''", '', '', 'The output should be:', '', 'What dividends were declared on September 20, 2022?', 'What dividend announcements were made on September 20, 2022?', "What were the dividend payments made on September 20, 2022?''", '', '', "This is an example of how you can generate alternative questions by changing the wording, using synonyms, or rephrasing the que

In [ ]:
print(len(unique_docs))
print(unique_docs)

36
[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/data/RAG_input/Microsoft_2023_Annual_Report.pdf', 'page': 1}, page_content='infusing this technology across every layer of the tech stack. And, finally, we will continue to drive operating leverage, \naligning our cost structure with our revenue growth.  \nAs we make progress on these priorities, we delivered strong results in fiscal year 2023, including a record $211  billion in \nrevenue and over $88  billion in operating income.  \nA NEW ERA OF AI  \nThere are two breakthroughs coming together to define this new era of AI. The first is the most universal interface: natural \nlanguage. The long arc of computing has, in many ways, been shaped by the pursuit of increasingly intuitive human -\ncomput er interfaces —keyboards, mice, touch screens. We believe we have now arrived at the next big step forward —\nnatural language —and will quickly go beyond, to see, hear, interpret, and make sense of our intent and the 

In [ ]:
for item in unique_docs:
  print(item)
  print("---------------")

page_content='infusing this technology across every layer of the tech stack. And, finally, we will continue to drive operating leverage, 
aligning our cost structure with our revenue growth.  
As we make progress on these priorities, we delivered strong results in fiscal year 2023, including a record $211  billion in 
revenue and over $88  billion in operating income.  
A NEW ERA OF AI  
There are two breakthroughs coming together to define this new era of AI. The first is the most universal interface: natural 
language. The long arc of computing has, in many ways, been shaped by the pursuit of increasingly intuitive human -
comput er interfaces —keyboards, mice, touch screens. We believe we have now arrived at the next big step forward —
natural language —and will quickly go beyond, to see, hear, interpret, and make sense of our intent and the world around 
us.  
  
The second is the emerg ence of a powerful new reasoning engine. For years, we’ve digitized daily life, places, and thin

In [ ]:
output = multiquery_retriever_from_llm.get_relevant_documents(query=query)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
INFO:langchain.retrievers.multi_query:Generated queries: ['You are an AI language model assistant. Your task is ', '    to generate 3 different versions of the given user ', '    question to retrieve relevant documents from a vector  database. ', '    By generating multiple perspectives on the user question, ', '    your goal is to help the user overcome some of the limitations ', '    of distance-based similarity search. Provide these alternative ', "    questions separated by newlines. Original question: What was the dividend declared on September 20, 2022?''.", '', '    Here is the solution:', '    ```', '    What dividends were declared on Septemb

In [ ]:
for item in output:
  print(item)
  print("------------------------")

page_content='infusing this technology across every layer of the tech stack. And, finally, we will continue to drive operating leverage, \naligning our cost structure with our revenue growth.  \nAs we make progress on these priorities, we delivered strong results in fiscal year 2023, including a record $211  billion in \nrevenue and over $88  billion in operating income.  \nA NEW ERA OF AI  \nThere are two breakthroughs coming together to define this new era of AI. The first is the most universal interface: natural \nlanguage. The long arc of computing has, in many ways, been shaped by the pursuit of increasingly intuitive human -\ncomput er interfaces —keyboards, mice, touch screens. We believe we have now arrived at the next big step forward —\nnatural language —and will quickly go beyond, to see, hear, interpret, and make sense of our intent and the world around \nus.  \n  \nThe second is the emerg ence of a powerful new reasoning engine. For years, we’ve digitized daily life, place

### Rerank with mutiquery

In [ ]:
# !pip install FlashRank==0.2.6

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from flashrank import Ranker, RerankRequest

In [ ]:
ranker = Ranker()

ms-marco-TinyBERT-L-2-v2.zip: 100%|██████████| 3.26M/3.26M [00:00<00:00, 36.3MiB/s]


In [ ]:
passages = [{'text':c.page_content} for c in unique_docs]

In [ ]:
passages_1 = [{'text':c.page_content} for c in output]

In [ ]:
print(passages)

[{'text': 'infusing this technology across every layer of the tech stack. And, finally, we will continue to drive operating leverage, \naligning our cost structure with our revenue growth.  \nAs we make progress on these priorities, we delivered strong results in fiscal year 2023, including a record $211  billion in \nrevenue and over $88  billion in operating income.  \nA NEW ERA OF AI  \nThere are two breakthroughs coming together to define this new era of AI. The first is the most universal interface: natural \nlanguage. The long arc of computing has, in many ways, been shaped by the pursuit of increasingly intuitive human -\ncomput er interfaces —keyboards, mice, touch screens. We believe we have now arrived at the next big step forward —\nnatural language —and will quickly go beyond, to see, hear, interpret, and make sense of our intent and the world around \nus.  \n  \nThe second is the emerg ence of a powerful new reasoning engine. For years, we’ve digitized daily life, places, 

In [ ]:
print(passages_1)

[{'text': 'infusing this technology across every layer of the tech stack. And, finally, we will continue to drive operating leverage, \naligning our cost structure with our revenue growth.  \nAs we make progress on these priorities, we delivered strong results in fiscal year 2023, including a record $211  billion in \nrevenue and over $88  billion in operating income.  \nA NEW ERA OF AI  \nThere are two breakthroughs coming together to define this new era of AI. The first is the most universal interface: natural \nlanguage. The long arc of computing has, in many ways, been shaped by the pursuit of increasingly intuitive human -\ncomput er interfaces —keyboards, mice, touch screens. We believe we have now arrived at the next big step forward —\nnatural language —and will quickly go beyond, to see, hear, interpret, and make sense of our intent and the world around \nus.  \n  \nThe second is the emerg ence of a powerful new reasoning engine. For years, we’ve digitized daily life, places, 

In [ ]:
rerankrequest = RerankRequest(query=query, passages=passages)

In [ ]:
rerankrequest_1 = RerankRequest(query=query, passages=passages_1)

In [ ]:
results = ranker.rerank(rerankrequest)

In [ ]:
results_1 = ranker.rerank(rerankrequest_1)

In [ ]:
query

'What was the dividend declared on September 20, 2022?'

In [ ]:
results

[{'text': 'Declaration Date  Record Date  Payment  Date  Dividend  \nPer Share  Amount    \n          \nFiscal Year 2023        (In millions)  \n          \nSeptember  20, 2022  November  17, 2022  December  8, 2022  $ 0.68  $ 5,066   \nNovember  29, 2022  February  16, 2023  March  9, 2023   0.68   5,059   \nMarch  14, 2023  May 18, 2023  June  8, 2023   0.68   5,054   \nJune  13, 2023  August  17, 2023  September  14, 2023   0.68   5,054   \n  \nTotal      $ 2.72  $ 20,233   \n          \n          \nFiscal Year 2022          \n          \nSeptember  14, 2021  November  18, 2021  December  9, 2021  $ 0.62  $ 4,652   \nDecember  7, 2021  February  17, 2022  March  10, 2022   0.62   4,645   \nMarch  14, 2022  May 19, 2022  June  9, 2022   0.62   4,632   \nJune  14, 2022  August  18, 2022  September  8, 2022   0.62   4,621   \n  \n          \nTotal      $  2.48  $   18,550   \n          \nThe dividend declared on June  13, 2023 was included in other current liabilities as of June  30, 2

In [ ]:
results_1

[{'text': 'Declaration Date  Record Date  Payment  Date  Dividend  \nPer Share  Amount    \n          \nFiscal Year 2023        (In millions)  \n          \nSeptember  20, 2022  November  17, 2022  December  8, 2022  $ 0.68  $ 5,066   \nNovember  29, 2022  February  16, 2023  March  9, 2023   0.68   5,059   \nMarch  14, 2023  May 18, 2023  June  8, 2023   0.68   5,054   \nJune  13, 2023  August  17, 2023  September  14, 2023   0.68   5,054   \n  \nTotal      $ 2.72  $ 20,233   \n          \n          \nFiscal Year 2022          \n          \nSeptember  14, 2021  November  18, 2021  December  9, 2021  $ 0.62  $ 4,652   \nDecember  7, 2021  February  17, 2022  March  10, 2022   0.62   4,645   \nMarch  14, 2022  May 19, 2022  June  9, 2022   0.62   4,632   \nJune  14, 2022  August  18, 2022  September  8, 2022   0.62   4,621   \n  \n          \nTotal      $  2.48  $   18,550   \n          \nThe dividend declared on June  13, 2023 was included in other current liabilities as of June  30, 2

In [ ]:
print(query)
print("Best Output match: ")
print(results[0]["text"])

What was the dividend declared on September 20, 2022?
Best Output match: 
Declaration Date  Record Date  Payment  Date  Dividend  
Per Share  Amount    
          
Fiscal Year 2023        (In millions)  
          
September  20, 2022  November  17, 2022  December  8, 2022  $ 0.68  $ 5,066   
November  29, 2022  February  16, 2023  March  9, 2023   0.68   5,059   
March  14, 2023  May 18, 2023  June  8, 2023   0.68   5,054   
June  13, 2023  August  17, 2023  September  14, 2023   0.68   5,054   
  
Total      $ 2.72  $ 20,233   
          
          
Fiscal Year 2022          
          
September  14, 2021  November  18, 2021  December  9, 2021  $ 0.62  $ 4,652   
December  7, 2021  February  17, 2022  March  10, 2022   0.62   4,645   
March  14, 2022  May 19, 2022  June  9, 2022   0.62   4,632   
June  14, 2022  August  18, 2022  September  8, 2022   0.62   4,621   
  
          
Total      $  2.48  $   18,550   
          
The dividend declared on June  13, 2023 was included in oth

## RAG with MutiQuery

Initialize chain

In [ ]:
time_start = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=llm_model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        # max_length=1024,
        max_new_tokens=5000,
        device_map="auto",)
time_end = time()
print(f"Prepare pipeline: {round(time_end-time_start, 3)} sec.")

Prepare pipeline: 0.001 sec.


In [ ]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
multiquery_retriever_from_llm = MultiQueryRetriever.from_llm(
    llm=llm,
    retriever=retriever
)

In [ ]:
qa_multiquery = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [ ]:
query = "What was the dividend declared on September 20, 2022?"

In [ ]:
unique_docs = multiquery_retriever_from_llm.invoke(query)
print(len(unique_docs))
for item in unique_docs:
  print(item)
  print("---------------")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


11
page_content='infusing this technology across every layer of the tech stack. And, finally, we will continue to drive operating leverage, 
aligning our cost structure with our revenue growth.  
As we make progress on these priorities, we delivered strong results in fiscal year 2023, including a record $211  billion in 
revenue and over $88  billion in operating income.  
A NEW ERA OF AI  
There are two breakthroughs coming together to define this new era of AI. The first is the most universal interface: natural 
language. The long arc of computing has, in many ways, been shaped by the pursuit of increasingly intuitive human -
comput er interfaces —keyboards, mice, touch screens. We believe we have now arrived at the next big step forward —
natural language —and will quickly go beyond, to see, hear, interpret, and make sense of our intent and the world around 
us.  
  
The second is the emerg ence of a powerful new reasoning engine. For years, we’ve digitized daily life, places, and t

In [ ]:
full_response = test_rag(qa_multiquery, query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.
Question: What was the dividend declared on September 20, 2022?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Declaration Date  Record Date  Payment  Date  Dividend  
Per Share  Amount    
          
Fiscal Year 2023        (In millions)  
          
September  20, 2022  November  17, 2022  December  8, 2022  $ 0.68  $ 5,066   
November  29, 2022  February  16, 2023  March  9, 2023   0.68   5,059   
March  14, 2023  May 18, 2023  June  8, 2023   0.68   5,054   
June  13, 2023  August  17, 2023  September  14, 2023   0.68   5,054   
  
Total      $ 2.72  $ 20,233   
          
          
Fiscal Year 2022          
          
September  14, 2021  November  18, 2021  December  9, 2021  $ 0.62  $ 4,652   
December  7, 2021  February  17, 2022  March  10, 2022   0.62   4,645   
March  14, 2022  May 19, 2022  Ju

### Use custom chain

In [ ]:
!pip install langchain-core==0.2.12

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.output_parser import StrOutputParser
from langchain.schema.runnable  import RunnablePassthrough
from langchain.schema  import StrOutputParser
# from langchain_core.runnable import RunnableLambda, RunnablePassthrough

In [ ]:
import textwrap

def pprint_docs(docs):
    print(f"\n{'-' * 70}\n".join([f"Document {i+1}:\n\n" + "\n".join(textwrap.wrap(d.page_content)) for i, d in enumerate(docs)]))

def pprint_result(result):
    print("Answer: " + "\n".join(textwrap.wrap(result)))

In [ ]:
prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
Context: {context}
Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

In [ ]:
def do_retrieval(chain, question):
    # for i in range(len(questions)):
    #     print("-" * 40)
    #     print(f"Question: {questions[i]}\n")
    #     with get_openai_callback() as cb:
    #         pprint_result(chain.invoke(questions[i]))
    #         print(f'\nTotal Tokens: {cb.total_tokens}\n')
    print("-" * 40)
    print(f"Question: {question}\n")
    pprint_result(chain.invoke(question))
    # print(f'\nTotal Tokens: {cb.total_tokens}\n')


In [ ]:
multi_chain = (
    {"context": multiquery_retriever_from_llm, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
do_retrieval(multi_chain, query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


----------------------------------------
Question: Do we have any lease?



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer: Human:  Answer the question based only on the supplied context. If you
don't know the answer, say you don't know the answer. Context:
[Document(metadata={'page': 1, 'source': '/content/drive/MyDrive/Colab
Notebooks/data/RAG_input/Microsoft_2023_Annual_Report.pdf'},
page_content='infusing this technology across every layer of the tech
stack. And, finally, we will continue to drive operating leverage,
\naligning our cost structure with our revenue growth.  \nAs we make
progress on these priorities, we delivered strong results in fiscal
year 2023, including a record $211  billion in \nrevenue and over $88
billion in operating income.  \nA NEW ERA OF AI  \nThere are two
breakthroughs coming together to define this new era of AI. The first
is the most universal interface: natural \nlanguage. The long arc of
computing has, in many ways, been shaped by the pursuit of
increasingly intuitive human -\ncomput er interfaces —keyboards, mice,
touch screens. We believe we have now arrived at

In [ ]:
query = "How was the stock performance?"
full_response = test_rag(qa_multiquery, query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.
Question: How was the stock performance?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

9 STOCK PERFORMANCE  
COMPARISON OF 5 YEAR CUMULATIVE TOTAL RETURN*  
Among Microsoft Corporation, the S&P 500 Index  
and the NASDAQ Computer Index  
  
  
 
  
  6/18 6/19 6/20 6/21 6/22 6/23   
Microsoft Corporation  $  100.00   $  138.07   $  212.34   $  285.40   $  272.82   $  365.24   
S&P 500   100.00    110.42    118.70    167.13    149.39    178.66   
NASDAQ Computer   100.00    106.10    156.93    236.08    184.53    242.82   
* $100 invested on 6/30/18 in stock or index, including reinvestment of dividends. Fiscal year ending June 30.

9 STOCK PERFORMANCE  
COMPARISON OF 5 YEAR CUMULATIVE TOTAL RETURN*  
Among Microsoft Corporation, the S&P 500 Index  
and the NASDAQ Computer Index  
  
  
 
  
  6/18 6/19 6/2

In [ ]:
query = "Can you please show me the stock performance comparison graph among the competitor comapanies and describe it?"
full_response = test_rag(qa_multiquery, query)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.
Question: Can you please show me the stock performance comparison graph among the competitor comapanies and describe it?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

9 STOCK PERFORMANCE  
COMPARISON OF 5 YEAR CUMULATIVE TOTAL RETURN*  
Among Microsoft Corporation, the S&P 500 Index  
and the NASDAQ Computer Index  
  
  
 
  
  6/18 6/19 6/20 6/21 6/22 6/23   
Microsoft Corporation  $  100.00   $  138.07   $  212.34   $  285.40   $  272.82   $  365.24   
S&P 500   100.00    110.42    118.70    167.13    149.39    178.66   
NASDAQ Computer   100.00    106.10    156.93    236.08    184.53    242.82   
* $100 invested on 6/30/18 in stock or index, including reinvestment of dividends. Fiscal year ending June 30.

9 STOCK PERFORMANCE  
COMPARISON OF 5 YEAR CUMULATIVE TOTAL RETURN*  
Among Microsoft Corporation

In [ ]:
query = "Do we have any lease?"
full_response = test_rag(qa_multiquery, query)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.
Question: Do we have any lease?
Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

(In millions)      
        
Year Ended June  30, 2023  2022  2021  
        
Cash paid for amounts included in the measurement of lease liabilities:        
Operating cash flows from operating leases  $  2,706   $  2,368   $  2,052   
Operating cash flows from finance leases   501   429   386  
Financing cash flows from finance leases   1,056    896   648  
        
Right -of-use assets obtained in exchange for lease obligations:        
Operating leases   3,514    5,268    4,380   
Finance leases   3,128    4,234    3,290

(In millions)      
        
Year Ended June  30, 2023  2022  2021  
        
Cash paid for amounts included in the measurement of lease liabilities:        
Operating cash flows from operating leases  $  2,7

## Parent Retriver

In [ ]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

In [ ]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)

In [ ]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

In [ ]:
# The vectorstore to use to index the child chunks
# vectorstore = Chroma(
#     collection_name="split_parents", embedding_function=OpenAIEmbeddings()
# )

In [ ]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=embeddings
)

In [ ]:
# The storage layer for the parent documents
store = InMemoryStore()

In [ ]:
# The storage layer for the parent documents
store_full = InMemoryStore()

In [ ]:
# In this mode, we want to retrieve the full documents. Therefore, we only specify a child splitter.
full_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store_full,
    child_splitter=child_splitter,
    # parent_splitter=parent_splitter,
)

In [ ]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [ ]:
full_document_retriever.add_documents(documents, ids=None)

In [ ]:
len(list(store_full.yield_keys()))

85

There are 85 documents as the pdf has 85 pages.

In [ ]:
parent_document_retriever.add_documents(documents, ids=None)

In [ ]:
len(list(store.yield_keys()))

196

We can see that there are much more than 85 documents now - these are the larger chunks.

Let's make sure the underlying vector store still retrieves the small chunks.

In [ ]:
sub_docs = vectorstore.similarity_search("What was the dividend declared on September 20, 2022?")

In [ ]:
print(sub_docs[0].page_content)

December  7, 2021  February  17, 2022  March  10, 2022   0.62   4,645   
March  14, 2022  May 19, 2022  June  9, 2022   0.62   4,632   
June  14, 2022  August  18, 2022  September  8, 2022   0.62   4,621   
  
          
Total      $    2.48   $  18,550   
          
The dividend declared on June  13, 2023 was included in other current liabilities as of June  30, 2023.


In [ ]:
retrieved_docs = retriever.invoke("What was the dividend declared on September 20, 2022?")

In [ ]:
len(retrieved_docs[0].page_content)

1544

In [ ]:
print(retrieved_docs[0].page_content)

repurchased during the second qu arter of fiscal year 2022 were under the share repurchase programs approved on both 
September  14, 2021 and September  18, 2019. All other shares repurchased were under the share repurchase program 
approved on September  18, 2019. The above table excludes sha res repurchased to settle employee tax withholding related 
to the vesting of stock awards of $3.8  billion, $4.7  billion, and $4.4  billion for fiscal years 2023, 2022, and 2021, respectively.  
  
Dividends  
Our Board of Directors declared the following divid ends:  
  
Declaration Date  Record Date  Payment  Date  Dividend  
Per Share  Amount    
          
Fiscal Year 2023        (In millions)  
          
September  20, 2022  November  17, 2022  December  8, 2022  $ 0.68  $ 5,066   
November  29, 2022  February  16, 2023  March  9, 2023   0.68   5,059   
March  14, 2023  May 18, 2023  June  8, 2023   0.68   5,054   
June  13, 2023  August  17, 2023  September  14, 2023   0.68   5,054   
  